<a href='http://www.holoviews.org'><img src="../notebooks/assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Exercise 5: Exporting and Deploying Apps</h2></div>

In [ ]:
import time
from itertools import cycle

import holoviews as hv 
import geoviews as gv
import param
import parambokeh
import dask.dataframe as dd
import pandas as pd

from colorcet import cm_n
from bokeh.document import Document
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY

hv.extension('bokeh')

def taxi_trips_stream(source='../data/nyc_taxi_wide.parq', frequency='H'):
    """Generate dataframes grouped by given frequency"""
    def get_group(resampler, key):
        try:
            df = resampler.get_group(key)
            df.reset_index(drop=True)
        except KeyError:
            df = pd.DataFrame()
        return df

    df = pd.read_parquet(source,
                     columns=['tpep_pickup_datetime', 'pickup_x', 'pickup_y', 'tip_amount'])
    df = df.set_index('tpep_pickup_datetime', drop=True)
    df = df.sort_index()
    r = df.resample(frequency)
    chunks = [get_group(r, g) for g in sorted(r.groups)]
    indices = cycle(range(len(chunks)))
    while True:
        yield chunks[next(indices)]

trips = taxi_trips_stream()
example = next(trips)

## Example 1

In this example we will be doing something a bit more complex - building a full app with widgets. Below we have already outlined the basic components of this app, including a ``NYCTaxiStream`` class that takes some ``data`` along with the ``x_range`` and ``y_range`` and turns those into a set of ``datashade``d ``Points``. You should execute that cell before you change anything about it, so that you are sure it works at the start.

Your task will be to add some custom widgets to this plot by adding parameters to the ``NYCTaxiStream`` class to control the NYCTaxiStream output. 

1.) Begin by adding a widget that will allow us to select between the different cmaps available from ``cm_n.values()``.

<b><a href="#hint1" data-toggle="collapse">Hint</a></b>

<div id="hint1" class="collapse">
Adding a widget is as simple as adding a Parameter to the class. To select between different options use:
<br><br>
<code>
    param.ObjectSelector(default=..., objects=...) 
</code>
</div>

<b><a href="#hint2" data-toggle="collapse">Hint</a></b>

<div id="hint2" class="collapse">
The ``datashade`` operation accepts a ``cmap`` argument.
</div>

2.) Now add a parameter that filters the data based on one of the columns in the data, e.g. trips with >N passengers.


<b><a href="#hint3" data-toggle="collapse">Hint</a></b>

<div id="hint3" class="collapse">
The ``param.Number`` and ``param.NumericTuple`` parameters allow defining scalar values and ranges respectively. Ensure that you set the ``bounds`` which declare the range of valid values.
<br><br>
<code>
    param.ObjectSelector(default=..., objects=...) 
</code>
</div>

In [ ]:
class NYCTaxiStream(hv.streams.Stream):

    def make_view(self, data, x_range, y_range, **kwargs):
        points = hv.Points(data, ['pickup_x', 'pickup_y'])
        return datashade(points, x_range=x_range, y_range=y_range,
                         dynamic=False)

tile_options = dict(width=600,height=400,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png').opts(plot=dict(tile_options))

nyc_stream = NYCTaxiStream()
buffer = hv.streams.Buffer(example, length=1000000)
obj = tiles * hv.DynamicMap(nyc_stream.make_view, streams=[nyc_stream, RangeXY(), buffer])

plot = hv.renderer('bokeh').get_plot(obj, Document())
parambokeh.Widgets(nyc_stream, view_position='right', callback=nyc_stream.event, plots=[plot])

To test the app you can ``start`` and ``stop`` streaming updates by running the two cells below:

In [ ]:
from tornado.ioloop import PeriodicCallback

def update():
    buffer.send(next(trips))

periodic = PeriodicCallback(update, 100)
periodic.start()

In [ ]:
periodic.stop()

<b><a href="#solution1" data-toggle="collapse">Solution</a></b>

<div id="solution1" class="collapse">
<br>
<code>class NYCTaxiStream(hv.streams.Stream):

    colormap = param.ObjectSelector(default=cm_n["fire"], objects=cm_n.values())

    minimum_tip = param.Number(default=0, bounds=(0, 20))
    
    def make_view(self, data, x_range, y_range, **kwargs):
        points = hv.Points(data, ['pickup_x', 'pickup_y'])
        if self.minimum_tip:
            points = points.select(tip_amount=(self.minimum_tip, None))
        return datashade(points, x_range=x_range, y_range=y_range,
                         dynamic=False, cmap=self.colormap)
</div>

## Example 2

Take the app you have written move it to a ``.py`` file and adapt it so it can be deployed using bokeh server.